In [44]:
import numpy as np
import pandas as pd
from certa.eval import expl_eval
from certa.explain import explain
from certa.utils import merge_sources
from models.utils import from_type

In [45]:
dataset = 'amazon_google'
model_type = 'deeper'
model = from_type(model_type)
model.load('models/' + model_type + '/' + dataset)

* Costruzione indice degli embeddings.....Fatto. 400001 embeddings totali.
* Creazione del modello per il calcolo degli embeddings....
* Inizializzo il tokenizzatore.....Fatto: 400001 parole totali.
* Preparazione della matrice di embedding.....Fatto. Dimensioni matrice embeddings: (400002, 50)

°°° EMBEDDING MODEL °°°
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Tupla_A (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
Tupla_B (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding_lookup (Embedding)    (None, None, 50)     20000100    Tup

In [46]:
def predict_fn(x):
    return model.predict(x)

In [47]:
datadir = 'datasets/' + dataset
lsource = pd.read_csv(datadir + '/tableA.csv')
rsource = pd.read_csv(datadir + '/tableB.csv')
gt = pd.read_csv(datadir + '/train.csv')
valid = pd.read_csv(datadir + '/valid.csv')
test = pd.read_csv(datadir + '/test.csv')

In [49]:
test_df = merge_sources(test, 'ltable_', 'rtable_', lsource, rsource, ['label'], [])

In [108]:
idx = 46
rand_row = test_df.iloc[idx]
l_id = int(rand_row['ltable_id'])
l_tuple = lsource.iloc[l_id]
r_id = int(rand_row['rtable_id'])
r_tuple = rsource.iloc[r_id]
rand_row.head()

label                                           1
ltable_id                                      22
ltable_manufacturer    global-software-publishing
ltable_price                                 9.99
ltable_title            musicalis guitar workshop
Name: 46, dtype: object

In [109]:
l_tuple

id                                      22
title            musicalis guitar workshop
manufacturer    global-software-publishing
price                                 9.99
Name: 22, dtype: object

In [110]:
r_tuple

id                                             1435
title           musicalis universal guitar workshop
manufacturer             global-software-publishing
price                                          12.9
Name: 1435, dtype: object

In [111]:
num_triangles = 100
fast = False
max_predict = -1

In [112]:
saliency_df, cf_summary, counterfactual_examples, triangles = explain(l_tuple, r_tuple, lsource, rsource, 
                                                                      predict_fn, datadir, 
                                                                      num_triangles=num_triangles,
                                                                      fast=fast, max_predict=max_predict)

In [113]:
saliency_df.to_dict()

{'rtable_price': {0: 0.23275862068965517},
 'rtable_manufacturer': {0: 0.15517241379310345},
 'rtable_title': {0: 0.04310344827586207},
 'ltable_title': {0: 0.02586206896551724},
 'ltable_manufacturer': {0: 0.014367816091954023},
 'ltable_price': {0: 0.014367816091954023}}

In [114]:
cf_summary

rtable_manufacturer/rtable_price    0.517241
dtype: float64

In [115]:
counterfactual_examples

,ltable_title,ltable_manufacturer,ltable_price,rtable_title,rtable_manufacturer,rtable_price,alteredAttributes,droppedValues,copiedValues,nomatch_score,match_score,attr_count
0,musicalis guitar workshop,global-software-publishing,9.99,musicalis universal guitar workshop,global-software-publishing,1562.06,"(rtable_price,)",[12.9],[1562.06],0.437370,0.562630,2
65,musicalis guitar workshop,global-software-publishing,9.99,musicalis universal guitar workshop,NaN,21.99,"(rtable_manufacturer, rtable_price)","[global-software-publishing, 12.9]","[nan, 21.99]",0.281680,0.718320,2
64,musicalis guitar workshop,global-software-publishing,9.99,musicalis universal guitar workshop,global-software-publishing,21.99,"(rtable_price,)",[12.9],[21.99],0.388520,0.611480,2
62,musicalis guitar workshop,global-software-publishing,9.99,musicalis universal guitar workshop,NaN,44.99,"(rtable_manufacturer, rtable_price)","[global-software-publishing, 12.9]","[nan, 44.99]",0.307953,0.692047,2
61,musicalis guitar workshop,global-software-publishing,9.99,musicalis universal guitar workshop,global-software-publishing,44.99,"(rtable_price,)",[12.9],[44.99],0.396448,0.603552,2
...,...,...,...,...,...,...,...,...,...,...,...,...
18,musicalis guitar workshop,global-software-publishing,9.99,allume anime studio 5,NaN,44.99,"(rtable_title, rtable_manufacturer, rtable_price)","[musicalis universal guitar workshop, global-s...","[allume anime studio 5, nan, 44.99]",0.325786,0.674214,3
69,musicalis guitar workshop,global-software-publishing,9.99,emedia music corp emedia piano & keyboard method,NaN,49.21,"(rtable_title, rtable_manufacturer, rtable_price)","[musicalis universal guitar workshop, global-s...",[emedia music corp emedia piano & keyboard met...,0.178296,0.821704,3
63,musicalis guitar workshop,global-software-publishing,9.99,allume anime studio 5,NaN,44.99,"(rtable_title, rtable_manufacturer, rtable_price)","[musicalis universal guitar workshop, global-s...","[allume anime studio 5, nan, 44.99]",0.325786,0.674214,3
9,musicalis guitar workshop,global-software-publishing,9.99,fogware hyb art dvd,NaN,21.99,"(rtable_title, rtable_manufacturer, rtable_price)","[musicalis universal guitar workshop, global-s...","[fogware hyb art dvd, nan, 21.99]",0.318725,0.681275,3
